<a href="https://colab.research.google.com/github/kazarezova/bioinformatics_for_NGS_ChIPseq/blob/main/kazarezova_ngs_ChIPseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2
Экстрагируйте риды и обрежьте их с помощью Trimmomatic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
for i in ['1', '2', 'control']:
  !cp ./drive/MyDrive/HSE_2022/UxuR_{i}.fastq UxuR_{i}.fastq

In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:19
🔁 Restarting kernel...


In [ ]:
!conda install -c bioconda Trimmomatic

In [5]:
# http://www.usadellab.org/cms/?page=trimmomatic
for i in ['1', '2', 'control']:
  !trimmomatic SE -phred33 UxuR_{i}.fastq UxuR_{i}.trim.fastq ILLUMINACLIP:TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36

TrimmomaticSE: Started with arguments:
 -phred33 UxuR_1.fastq UxuR_1.trim.fastq ILLUMINACLIP:TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36
Automatically using 2 threads
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
ILLUMINACLIP: Using 0 prefix pairs, 2 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Input Reads: 58606277 Surviving: 51471087 (87.83%) Dropped: 7135190 (12.17%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -phred33 UxuR_2.fastq UxuR_2.trim.fastq ILLUMINACLIP:TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36
Automatically using 2 threads
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
ILLUMINACLIP: Using 0 prefix pairs, 2 forward/reverse sequences, 0 forward only sequences, 0 reverse only seque

# 3
Наложите на геном E. coli K-12 MG1655 genome с помощью Bowtie

https://www.ncbi.nlm.nih.gov/genome/167?genome_assembly_id=161521

In [6]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
!gunzip GCF_000005845.2_ASM584v2_genomic.fna.gz

--2022-12-19 17:47:23--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.7, 2607:f220:41f:250::228, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379902 (1.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’

GCF_000005845.2_ASM 100%[===================>]   1.32M  8.27MB/s    in 0.2s    

2022-12-19 17:47:23 (8.27 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’ saved [1379902/1379902]



In [ ]:
!conda install -c bioconda bowtie

In [ ]:
!bowtie-build GCF_000005845.2_ASM584v2_genomic.fna index

In [2]:
for i in ['1', '2', 'control']:
  !bowtie  -p 2 -x index -q UxuR_{i}.trim.fastq -S UxuR_{i}.sam

# reads processed: 51471087
# reads with at least one alignment: 37288575 (72.45%)
# reads that failed to align: 14182512 (27.55%)
Reported 37288575 alignments
# reads processed: 23065214
# reads with at least one alignment: 20128419 (87.27%)
# reads that failed to align: 2936795 (12.73%)
Reported 20128419 alignments
# reads processed: 21894690
# reads with at least one alignment: 15285002 (69.81%)
# reads that failed to align: 6609688 (30.19%)
Reported 15285002 alignments


In [ ]:
!conda install -c bioconda samtools

In [4]:
for i in ['1', '2', 'control']:
  !samtools view -S -b UxuR_{i}.sam > UxuR_{i}.bam

In [5]:
# освободим место на диске
#for i in ['1', '2', 'control']:
#  !rm UxuR_{i}.sam
#  !rm UxuR_{i}.trim.fastq
#  !rm UxuR_{i}.fastq
#!rm index.*

# 4
Найдите пики с помощью MACS2 (подсказки тут - https://hbctraining.github.io/Intro-to-ChIPseq/lessons/05_peak_calling_macs.html) и сделайте картинку/табличку с генами, к которым они относятся

In [ ]:
!conda install -c bioconda macs2

In [ ]:
for i in ['1', '2']:
  !macs2 callpeak -t UxuR_{i}.bam \
	  -c UxuR_control.bam \
 	  -f BAM \
	  -n UxuR_{i} \
	  --outdir macs2 --nomodel --extsize 100 -p 0.05

In [8]:
# аннотация генома E. coli
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz
!gunzip GCF_000005845.2_ASM584v2_genomic.gff.gz

--2022-12-19 18:41:34--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 165.112.9.230, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433547 (423K) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.gff.gz’

GCF_000005845.2_ASM 100%[===================>] 423.39K  --.-KB/s    in 0.1s    

2022-12-19 18:41:35 (3.05 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.gff.gz’ saved [433547/433547]



In [28]:
import pandas as pd

!cat 'GCF_000005845.2_ASM584v2_genomic.gff' > 'EColi_ann.txt'

with open('EColi_ann.txt', 'r') as fin:
    data = fin.read().splitlines(True)
with open('EColi_ann.txt', 'w') as fout:
    fout.writelines(data[7:])

ecoli_ann = pd.DataFrame(pd.read_csv('EColi_ann.txt', sep='\t', names=['NC_000913.3', 'RefSeq', 'type', 'start', 'stop', '.', 'strand', '0', 'info']))

ecoli_ann.head()

,NC_000913.3,RefSeq,type,start,stop,.,strand,0,info
0,NC_000913.3,RefSeq,region,1.0,4641652.0,.,+,.,ID=NC_000913.3:1..4641652;Dbxref=taxon:511145;...
1,NC_000913.3,RefSeq,gene,190.0,255.0,.,+,.,"ID=gene-b0001;Dbxref=ASAP:ABE-0000006,ECOCYC:E..."
2,NC_000913.3,RefSeq,CDS,190.0,255.0,.,+,0,ID=cds-NP_414542.1;Parent=gene-b0001;Dbxref=Un...
3,NC_000913.3,RefSeq,gene,337.0,2799.0,.,+,.,"ID=gene-b0002;Dbxref=ASAP:ABE-0000008,ECOCYC:E..."
4,NC_000913.3,RefSeq,CDS,337.0,2799.0,.,+,0,ID=cds-NP_414543.1;Parent=gene-b0002;Dbxref=Un...


In [10]:
!cat macs2/'UxuR_1_peaks.xls' > 'UxuR_1_peaks.txt'

with open('UxuR_1_peaks.txt', 'r') as fin:
    data = fin.read().splitlines(True)
with open('UxuR_1_peaks.txt', 'w') as fout:
    fout.writelines(data[30:])

uxur1 = pd.DataFrame(pd.read_csv('UxuR_1_peaks.txt', sep='\t', names=['chr', 'start', 'end', 'length', 'abs_summit', 'pileup', '-LOG10(pvalue)', 'fold_enrichment', '-LOG10(qvalue)', 'name']))

uxur1.head()

,chr,start,end,length,abs_summit,pileup,-LOG10(pvalue),fold_enrichment,-LOG10(qvalue),name
0,NC_000913.3,313286,313385,100,313300,160.23,1.43806,1.15166,0,UxuR_1_peak_1
1,NC_000913.3,571027,571190,164,571111,144.21,1.73413,1.19023,0,UxuR_1_peak_2
2,NC_000913.3,579592,579965,374,579801,144.21,1.83591,1.20007,0,UxuR_1_peak_3
3,NC_000913.3,1215825,1215962,138,1215895,138.60,2.24074,1.24643,0,UxuR_1_peak_4
4,NC_000913.3,1586299,1586398,100,1586309,142.61,1.43987,1.16478,0,UxuR_1_peak_5


In [11]:
!cat macs2/'UxuR_2_peaks.xls' > 'UxuR_2_peaks.txt'

with open('UxuR_2_peaks.txt', 'r') as fin:
    data = fin.read().splitlines(True)
with open('UxuR_2_peaks.txt', 'w') as fout:
    fout.writelines(data[30:])

uxur2 = pd.DataFrame(pd.read_csv('UxuR_2_peaks.txt', sep='\t', names=['chr', 'start', 'end', 'length', 'abs_summit', 'pileup', '-LOG10(pvalue)', 'fold_enrichment', '-LOG10(qvalue)', 'name']))

uxur2.head()

,chr,start,end,length,abs_summit,pileup,-LOG10(pvalue),fold_enrichment,-LOG10(qvalue),name
0,NC_000913.3,236900,237007,108,236959,171.30,1.80390,1.18013,0,UxuR_2_peak_1
1,NC_000913.3,313045,313605,561,313306,180.41,3.57838,1.30511,0,UxuR_2_peak_2
2,NC_000913.3,568352,568514,163,568446,148.52,1.91999,1.20746,0,UxuR_2_peak_3
3,NC_000913.3,571019,571201,183,571197,147.61,1.34805,1.15201,0,UxuR_2_peak_4
4,NC_000913.3,571963,572100,138,572054,149.43,2.36487,1.24529,0,UxuR_2_peak_5


In [24]:
# пересечем пики двух повторов

summits_intersect = []

for summit in list(uxur1['abs_summit'].values):
  for start in list(uxur2['start'].values):
    end = uxur2.loc[uxur2['start']==start, 'end']
    if float(summit)>float(start) and float(summit)<float(end):
      summits_intersect.append(summit)

uxur = uxur1.loc[uxur1['abs_summit'].isin(summits_intersect)]
uxur # в пересечение попали все пики UxuR_1

,chr,start,end,length,abs_summit,pileup,-LOG10(pvalue),fold_enrichment,-LOG10(qvalue),name
0,NC_000913.3,313286,313385,100,313300,160.23,1.43806,1.15166,0,UxuR_1_peak_1
1,NC_000913.3,571027,571190,164,571111,144.21,1.73413,1.19023,0,UxuR_1_peak_2
2,NC_000913.3,579592,579965,374,579801,144.21,1.83591,1.20007,0,UxuR_1_peak_3
3,NC_000913.3,1215825,1215962,138,1215895,138.60,2.24074,1.24643,0,UxuR_1_peak_4
8,NC_000913.3,1699695,1699798,104,1699751,160.23,1.38657,1.14650,0,UxuR_1_peak_9
9,NC_000913.3,1776423,1776562,140,1776520,153.82,1.50793,1.16680,0,UxuR_1_peak_10
10,NC_000913.3,2102898,2103041,144,2102943,141.81,2.80736,1.29423,0,UxuR_1_peak_11
12,NC_000913.3,2105464,2105598,135,2105534,130.59,2.33040,1.26321,0,UxuR_1_peak_13
14,NC_000913.3,2990353,2990596,244,2990543,133.79,1.94853,1.22540,0,UxuR_1_peak_15
15,NC_000913.3,2991733,2991846,114,2991821,129.79,2.03350,1.23770,0,UxuR_1_peak_16


In [21]:
# найдем в какие гены попадают пики

genes = []
ecoli_genes = ecoli_ann.loc[ecoli_ann['type']=='gene']

for summit in list(uxur['abs_summit'].values):
  for start in list(ecoli_genes['start'].values):
    end = ecoli_genes.loc[ecoli_genes['start']==start, 'stop']
    try:
      if float(summit)>float(start) and float(summit)<float(end): # для + цепи
        genes.append(start)
      if float(summit)<float(start) and float(summit)>float(end): # для - цепи
        genes.append(start)
    except TypeError:
      continue

uxur_genes = ecoli_genes.loc[ecoli_genes['start'].isin(genes)]
uxur_genes

,NC_000913.3,RefSeq,type,start,stop,.,strand,0,info
1178,NC_000913.3,RefSeq,gene,570893.0,571444.0,.,+,.,"ID=gene-b0545;Dbxref=ASAP:ABE-0001869,ECOCYC:G..."
2496,NC_000913.3,RefSeq,gene,1215789.0,1216025.0,.,+,.,"ID=gene-b1164;Dbxref=ASAP:ABE-0003907,ECOCYC:G..."
3530,NC_000913.3,RefSeq,gene,1699355.0,1700947.0,.,+,.,"ID=gene-b1621;Dbxref=ASAP:ABE-0005429,ECOCYC:E..."
3690,NC_000913.3,RefSeq,gene,1775587.0,1777182.0,.,+,.,"ID=gene-b1694;Dbxref=ASAP:ABE-0005656,ECOCYC:E..."
4427,NC_000913.3,RefSeq,gene,2105065.0,2106057.0,.,-,.,"ID=gene-b2034;Dbxref=ASAP:ABE-0006751,ECOCYC:E..."
6158,NC_000913.3,RefSeq,gene,2992094.0,2993470.0,.,+,.,"ID=gene-b2852;Dbxref=ASAP:ABE-0009368,ECOCYC:G..."


In [22]:
uxur_genes['product'] = [0 for i in range(uxur_genes.shape[0])]

for info in list(uxur_genes['info'].values):
  info_str = str(info).split(';')
  for substring in info_str:
    if 'Name' in substring:
      uxur_genes.loc[uxur_genes['info']==info, 'product'] = substring[5:]

uxur_genes.drop(columns='info')

<ipython-input-22-e4361623e602>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uxur_genes['product'] = [0 for i in range(uxur_genes.shape[0])]
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,NC_000913.3,RefSeq,type,start,stop,.,strand,0,product
1178,NC_000913.3,RefSeq,gene,570893.0,571444.0,.,+,.,ybcL
2496,NC_000913.3,RefSeq,gene,1215789.0,1216025.0,.,+,.,ycgZ
3530,NC_000913.3,RefSeq,gene,1699355.0,1700947.0,.,+,.,malX
3690,NC_000913.3,RefSeq,gene,1775587.0,1777182.0,.,+,.,ydiF
4427,NC_000913.3,RefSeq,gene,2105065.0,2106057.0,.,-,.,wbbI
6158,NC_000913.3,RefSeq,gene,2992094.0,2993470.0,.,+,.,ygeH


# 5
На основании найденных пиков предположите, какие функции могут выполнять белки UxuR и YihW

https://www.ncbi.nlm.nih.gov/
- ybcL - periplasmic protein YbcL, may be a kinase regulator
- YcgZ - putative two-component system connector protein YcgZ
- malX - PTS enzyme IIBC component MalX
- ydiF - putative acetate-CoA transferase YdiF
- wbbI - beta-1,6-galactofuranosyltransferase WbbI that uses UDP-galactofuranose as the donor and has a modest preference for pyranoside acceptor substrates of the -D-gluco configuration
- ygeH - putative transcriptional regulator YgeH, a remnant of the type three secretion system (T3SS) pathogenicity island ETT2

Видимо, гены нашлись неправильно. UxuR является транскрипционным репрессором оперона UxuRBA, участвует в инактивации глюкуронового пути (пути уроновых кислот) https://pubchem.ncbi.nlm.nih.gov/gene/948849. Его пики должны были попасть в гены UxuA (D-mannonate dehydratase), UxuB (D-mannonate oxidoreductase) и прочие гены UxuR-регулона

# 6
Используя ChIPMunk, найдите мотив для выбранного белка

In [ ]:
!conda install -c bioconda bedtools

In [26]:
! bedtools getfasta -fi GCF_000005845.2_ASM584v2_genomic.fna -bed macs2/UxuR_1_peaks.narrowPeak > peaks.fa

ChIPMunk: Please save your ticket number aaaaaaabui to be able to view results later